# Title: Data Processing for Hurricane Impact Analysis

## Description
This script processes and cleans a dataset related to the economic and social impacts of hurricanes in the United States. It leverages the OpenCage Data API to obtain latitude and longitude data for various locations and implements caching to optimize API usage.

## Technical Overview
- **Asynchronous Requests**: Utilized aiohttp and asyncio to handle geocoding API calls concurrently, improving processing speed and efficiency.
- **OpenCage Data API**: Used to retrieve reliable geocoding data, translating location names into geographic coordinates.
- **Caching Mechanism**: Implemented a cache to store the results of previous API calls, reducing redundant requests and helping avoid API rate limits.
- **Nesting Asyncio**: Applied nest_asyncio to facilitate running asynchronous event loops in Jupyter Notebooks or similar environments.

## Resource References
- **OpenCage Data API**: [opencagedata.com](https://opencagedata.com/)
- **aiohttp** Documentation: [aiohttp.readthedocs.io](https://docs.aiohttp.org/en/stable/)
- **asyncio** Documentation: [docs.python.org/3/library/asyncio.html](https://docs.python.org/3/library/asyncio.html)
- **nest_asyncio**: [nest_asyncio GitHub Repository](https://github.com/erdewit/nest_asyncio)
- **Caching Strategies**: Information on Python caching strategies was sourced from various online articles and documentation.

## Code Overview
The script begins by importing necessary libraries, loading the initial dataset, and then processing the data to include latitude and longitude information using the OpenCage Data API. A caching mechanism is implemented to optimize API usage and improve efficiency.

In [1]:
import requests
import pandas as pd
from config import api_key
import aiohttp
import asyncio
import nest_asyncio
import json

In [2]:
# Load cache from a file (at the start of your script)
try:
    with open('location_cache.json', 'r') as cache_file:
        location_cache = json.load(cache_file)
except FileNotFoundError:
    location_cache = {}

In [3]:
# Read the CSV file
emdat_data = "Resource/public_emdat_data.csv"
emdat_data_df = pd.read_csv(emdat_data)

emdat_data_df

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,2000-0643-MEX,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Keith,MEX,...,NaN,NaN,NaN,NaN,1000.0,1769.0,56.514291,"[{""adm1_code"":2031,""adm1_name"":""Campeche""},{""a...",2005-12-16,2023-09-25
1,2000-0652-USA,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Leslie,USA,...,NaN,NaN,NaN,NaN,219000.0,387513.0,56.514291,"[{""adm2_code"":29003,""adm2_name"":""Broward""},{""a...",2003-07-01,2023-09-25
2,2001-0242-USA,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Allison,USA,...,NaN,NaN,3500000.0,6022907.0,6000000.0,10324983.0,58.111474,"[{""adm1_code"":3223,""adm1_name"":""Florida""},{""ad...",2006-10-11,2023-09-25
3,2001-0488-MEX,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Dalila,MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,58.111474,"[{""adm1_code"":2032,""adm1_name"":""Chiapas""},{""ad...",2003-07-01,2023-09-25
4,2001-0562-MEX,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Juliette,MEX,...,NaN,NaN,150000.0,258125.0,400000.0,688332.0,58.111474,"[{""adm1_code"":2030,""adm1_name"":""Baja Californi...",2004-10-28,2023-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2023-0675-MEX,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,GLIDE:TC-2023-000209,Tropical storm 'Otis',MEX,...,NaN,NaN,4000000.0,4000000.0,12000000.0,12000000.0,100.000000,NaN,2023-10-30,2024-03-04
142,2024-0402-MEX,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Tropical storm 'Alberto',MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-21,2024-06-24
143,2024-0422-CAN,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,GLIDE:TC-2024-000105,Hurricane 'Beryl',CAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-16,2024-08-01
144,2024-0422-USA,No,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,GLIDE:TC-2024-000105,Hurricane 'Beryl',USA,...,NaN,NaN,NaN,NaN,4500000.0,NaN,NaN,NaN,2024-07-10,2024-08-01


In [4]:
# Remove unnecessary columns to create a cleaned DataFrame
columns_to_keep = ["Start Year", "Start Month", "Start Day", "End Year", "End Month", "End Day", 
                   "Location", "Total Deaths", "No. Affected", 
                   "Total Affected", "Insured Damage ('000 US$)", "Total Damage ('000 US$)", 
                   "Magnitude", "Magnitude Scale"]
cleaned_emdat_data_df = emdat_data_df[columns_to_keep]
cleaned_emdat_data_df

,Start Year,Start Month,Start Day,End Year,End Month,End Day,Location,Total Deaths,No. Affected,Total Affected,Insured Damage ('000 US$),Total Damage ('000 US$),Magnitude,Magnitude Scale
0,2000,9,29,2000,10,3,"Puebla, Campeche, Quintana Roo, Yucatan, Verac...",23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph
1,2000,10,4,2000,10,4,"Miami-Dade, Monroe, Broward, Collier districts...",2.0,14418.0,17433.0,NaN,219000.0,NaN,Kph
2,2001,6,5,2001,6,17,"Texas, Mississippi, Louisiana, Florida, Pennsy...",41.0,102000.0,172000.0,3500000.0,6000000.0,NaN,Kph
3,2001,7,25,2001,7,25,"Guerrero, Chiapas provinces",NaN,100.0,100.0,NaN,NaN,NaN,Kph
4,2001,9,24,2001,10,2,Baja California Sur province,3.0,3000.0,3800.0,150000.0,400000.0,120.0,Kph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2023,10,22,2023,10,25,Acapulco (central Guerrero State),104.0,988000.0,988000.0,4000000.0,12000000.0,270.0,Kph
142,2024,6,20,2024,6,22,"Nuevo Leon, Veracruz, Tamaulipas, Coahuila,",4.0,NaN,NaN,NaN,NaN,NaN,Kph
143,2024,7,12,2024,7,12,Nouvelle Ecosse,1.0,NaN,NaN,NaN,NaN,NaN,Kph
144,2024,7,8,2024,7,9,"Texas, Louisiana",8.0,207.0,207.0,NaN,4500000.0,130.0,Kph


In [5]:
# Drop rows with missing values in any column
droped_emdat_data_df = cleaned_emdat_data_df.dropna()

droped_emdat_data_df

,Start Year,Start Month,Start Day,End Year,End Month,End Day,Location,Total Deaths,No. Affected,Total Affected,Insured Damage ('000 US$),Total Damage ('000 US$),Magnitude,Magnitude Scale
4,2001,9,24,2001,10,2,Baja California Sur province,3.0,3000.0,3800.0,150000.0,400000.0,120.0,Kph
8,2002,9,20,2002,9,20,"Campeche, Quintana Roo, Yucatan provinces",13.0,500000.0,500030.0,280000.0,640000.0,180.0,Kph
10,2002,9,26,2002,9,27,"Louisiane, Mississippi, Alabama, Tennessee pro...",1.0,13200.0,13200.0,200000.0,300000.0,105.0,Kph
14,2003,9,18,2003,9,22,"North Carolina, Maryland, Virginia, West Virgi...",16.0,225000.0,225000.0,1685000.0,3370000.0,170.0,Kph
19,2004,8,13,2004,8,13,"Bay, Calhoun, Escambia, Franklin, Gadsden, Gul...",10.0,30000.0,30000.0,7600000.0,16000000.0,230.0,Kph
27,2005,8,29,2005,9,19,"Mobile, Bayou La Batre, Dauphin Island, Coden ...",1833.0,500000.0,500000.0,60000000.0,125000000.0,280.0,Kph
29,2005,9,23,2005,10,1,"Louisiana, Texas, Mississippi provinces",10.0,300000.0,300000.0,11300000.0,16000000.0,280.0,Kph
32,2005,10,24,2005,10,24,"Florida Keys, Naples areas (Collier district, ...",4.0,30000.0,30000.0,10350000.0,14300000.0,165.0,Kph
38,2007,8,21,2007,8,24,"Yucatan, Quintana Roo, Campeche, Veracruz, Hid...",9.0,140000.0,140000.0,475000.0,600000.0,265.0,Kph
47,2008,9,12,2008,9,16,"Galveston, Brazoria, Harris, Chambers, Jeffers...",82.0,200000.0,200000.0,15000000.0,30000000.0,200.0,Kph


In [7]:
# Save the updated DataFrame to a new CSV file
#yearly_emdat_data_df.to_csv("Resource/yearly_emdat_data.csv", index=False)

print("Process completed and file saved.")

Process completed and file saved.


In [10]:
# Split the locations into separate rows and reset index
yearly_emdat_data_df = yearly_emdat_data_df.assign(Location=yearly_emdat_data_df['Location'].str.split(',')).explode('Location').reset_index(drop=True)


In [11]:
# Strip whitespace from locations
emdat_data_map_df['Location'] = cleaned_emdat_data_df['Location'].str.strip()
emdat_data_map_df

,Start Year,Start Month,Start Day,End Year,End Month,End Day,Location,Total Deaths,No. Affected,Total Affected,Insured Damage ('000 US$),Total Damage ('000 US$),Magnitude,Magnitude Scale
0,2000,9,29,2000,10,3,Puebla,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph
1,2000,9,29,2000,10,3,Campeche,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph
2,2000,9,29,2000,10,3,Quintana Roo,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph
3,2000,9,29,2000,10,3,Yucatan,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph
4,2000,9,29,2000,10,3,Veracruz,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,2024,7,12,2024,7,12,Nouvelle Ecosse,1.0,NaN,NaN,NaN,NaN,NaN,Kph
817,2024,7,8,2024,7,9,Texas,8.0,207.0,207.0,NaN,4500000.0,130.0,Kph
818,2024,7,8,2024,7,9,Louisiana,8.0,207.0,207.0,NaN,4500000.0,130.0,Kph
819,2024,6,30,2024,6,30,Veracruz,6.0,20000.0,20000.0,NaN,NaN,65.0,Kph


### Using Asynchronous Functions with aiohttp and asyncio

In this section, we define asynchronous functions to efficiently retrieve latitude and longitude data for each location. Traditional synchronous requests could significantly slow down the processing time, especially with large datasets. By implementing asynchronous requests, we can handle multiple requests concurrently, making the process much faster.

**Why Asynchronous?**
- **Performance**: Asynchronous requests allow multiple operations to occur at the same time, which reduces the waiting time for each request.
- **Learning Resource**: We learned about asynchronous programming through online documentation and resources like [aiohttp Documentation](https://docs.aiohttp.org/en/stable/) and [asyncio Documentation](https://docs.python.org/3/library/asyncio.html). This technique was not covered in traditional coursework, but it has significantly improved the efficiency of data processing in this project.

```python

In [12]:
# Define the base URL for the API
url = "https://api.opencagedata.com/geocode/v1/json?"

# Asynchronous function to get latitude and longitude for a given location
async def get_lat_long(session, location):
    request_url = f"{url}q={location}&key={api_key}"
    async with session.get(request_url) as response:
        if response.status == 200:
            data = await response.json()
            if data['results']:
                lat = data['results'][0]['geometry']['lat']
                long = data['results'][0]['geometry']['lng']
                return lat, long
        return None, None

# Asynchronous function to handle multiple requests
async def fetch_all_locations(df):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, row in df.iterrows():
            task = asyncio.create_task(get_lat_long(session, row['Location']))  # Changed to create_task
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        return responses

In [13]:
# Create new columns for latitude and longitude if they don't exist
if 'Latitude' not in emdat_data_map_df.columns:
    emdat_data_map_df['Latitude'] = None
if 'Longitude' not in emdat_data_map_df.columns:
    emdat_data_map_df['Longitude'] = None

In [14]:
# Allow nested asyncio event loops (necessary for Jupyter Notebook)
nest_asyncio.apply()

In [15]:
# Run the asynchronous fetching
results = asyncio.run(fetch_all_locations(emdat_data_map_df))

# Assign results back to the DataFrame
for idx, (lat, long) in enumerate(results):
    emdat_data_map_df.at[idx, 'Latitude'] = lat
    emdat_data_map_df.at[idx, 'Longitude'] = long

emdat_data_map_df

,Start Year,Start Month,Start Day,End Year,End Month,End Day,Location,Total Deaths,No. Affected,Total Affected,Insured Damage ('000 US$),Total Damage ('000 US$),Magnitude,Magnitude Scale,Latitude,Longitude
0,2000,9,29,2000,10,3,Puebla,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph,18.833333,-98.0
1,2000,9,29,2000,10,3,Campeche,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph,19.0,-90.5
2,2000,9,29,2000,10,3,Quintana Roo,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph,19.666667,-88.5
3,2000,9,29,2000,10,3,Yucatan,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph,20.684596,-88.875567
4,2000,9,29,2000,10,3,Veracruz,23.0,30000.0,30000.0,NaN,1000.0,150.0,Kph,19.333333,-96.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,2024,7,12,2024,7,12,Nouvelle Ecosse,1.0,NaN,NaN,NaN,NaN,NaN,Kph,45.19604,-63.165379
817,2024,7,8,2024,7,9,Texas,8.0,207.0,207.0,NaN,4500000.0,130.0,Kph,31.26389,-98.545612
818,2024,7,8,2024,7,9,Louisiana,8.0,207.0,207.0,NaN,4500000.0,130.0,Kph,30.870388,-92.007126
819,2024,6,30,2024,6,30,Veracruz,6.0,20000.0,20000.0,NaN,NaN,65.0,Kph,19.333333,-96.666667


In [16]:
# Drop rows with missing values in any column
emdat_data_map_df = emdat_data_map_df.dropna()

emdat_data_map_df

,Start Year,Start Month,Start Day,End Year,End Month,End Day,Location,Total Deaths,No. Affected,Total Affected,Insured Damage ('000 US$),Total Damage ('000 US$),Magnitude,Magnitude Scale,Latitude,Longitude
35,2002,9,20,2002,9,20,Campeche,13.0,500000.0,500030.0,280000.0,640000.0,180.0,Kph,19.0,-90.5
36,2002,9,20,2002,9,20,Quintana Roo,13.0,500000.0,500030.0,280000.0,640000.0,180.0,Kph,19.666667,-88.5
39,2002,9,26,2002,9,27,Louisiane,1.0,13200.0,13200.0,200000.0,300000.0,105.0,Kph,30.870388,-92.007126
40,2002,9,26,2002,9,27,Mississippi,1.0,13200.0,13200.0,200000.0,300000.0,105.0,Kph,32.971528,-89.73485
41,2002,9,26,2002,9,27,Alabama,1.0,13200.0,13200.0,200000.0,300000.0,105.0,Kph,33.258882,-86.829534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,2021,8,28,2021,9,2,Connecticut,96.0,14000.0,14000.0,36000000.0,65000000.0,240.0,Kph,41.65002,-72.734216
772,2021,8,28,2021,9,2,Virginia,96.0,14000.0,14000.0,36000000.0,65000000.0,240.0,Kph,37.123224,-78.492772
773,2021,8,28,2021,9,2,Pennsylvania,96.0,14000.0,14000.0,36000000.0,65000000.0,240.0,Kph,40.969989,-77.727883
774,2021,8,28,2021,9,2,Delaware,96.0,14000.0,14000.0,36000000.0,65000000.0,240.0,Kph,38.692045,-75.401331


In [17]:
# Save the updated DataFrame to a new CSV file
#emdat_data_map_df.to_csv("Resource/emdat_data_map.csv", index=False)

print("Process completed and file saved.")

Process completed and file saved.
